In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 10.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=4fb5efb5503403bd5faf41f45defa551aa9704cfe23e0a15b9888b2ef59fecc8
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_review_Analysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...|2015-08-31 00:00:00|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva L

In [4]:
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|                Y|
|R1L94ESVC657A9|          5|            0|          0|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|                Y|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|     

In [28]:
total_votes_df = vine_df.filter("total_votes >= 20")
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
|R1M8EEGH2TVS0Q|          1|            6|         29|   N|                Y|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|     

In [50]:
import pyspark.sql.functions as fun
helpful_votes_df = total_votes_df.withColumn("more_helpful",(fun.col("helpful_votes") / fun.col("total_votes"))).filter("more_helpful >= 0.5")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      more_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|0.8043478260869565|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|0.9646464646464646|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|              0.98|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|0.9642857142857143|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|               1.0|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|0.9523809523809523|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|0.9292035398230089|
|R3JF2SBRHZITRZ|          5|  

In [68]:
# Challenge asked vor "== 'Y' but this code also works * paid_df = helpful_votes_df.filter(fun.col("vine").contains('Y'))
paid_df = helpful_votes_df.filter(fun.col("vine") =='Y')
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      more_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RG70K1HMY4LXX|          4|           37|         44|   Y|                N|0.8409090909090909|
|R1DHGTNXDXJ0GB|          5|           78|         93|   Y|                N|0.8387096774193549|
|R3B5S3OSA8N6AO|          5|           38|         43|   Y|                N|0.8837209302325582|
|R29X0YBS0SBIUP|          5|           56|         74|   Y|                N|0.7567567567567568|
|R1ZEDMCU6NYC8Q|          3|           26|         27|   Y|                N|0.9629629629629629|
| RVBJM1LUI9X9K|          5|           27|         28|   Y|                N|0.9642857142857143|
| R1KASHD9QSDMV|          4|           18|         21|   Y|                N|0.8571428571428571|
|R1M3LATUI38MK0|          5|  

In [72]:
unpaid_df = helpful_votes_df.filter(fun.col("vine") =='N')
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      more_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|0.8043478260869565|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|0.9646464646464646|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|              0.98|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|0.9642857142857143|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|               1.0|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|0.9523809523809523|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|0.9292035398230089|
|R3JF2SBRHZITRZ|          5|  

In [79]:
reviews_df = paid_df.count()
print(f'There are {reviews_df} paid reviews')

There are 497 paid reviews


In [80]:
unpaid_reviews_df = unpaid_df.count()
print(f'There are {unpaid_reviews_df} unpaid reviews')

There are 120863 unpaid reviews


In [82]:
p = paid_df.filter(fun.col("Star_rating") == 5).count()
print(f'There are {p} paid 5 star ratings')

There are 220 paid 5 star ratings


In [83]:
u = unpaid_df.filter(fun.col("star_rating") ==5).count()
print(f'There are {u} unpaid 5 star ratings')

There are 74470 unpaid 5 star ratings


In [93]:
percent_paid = (p / reviews_df) *100
print(f'{round(percent_paid, 2)}% of paid reviews had a 5 star rating')

44.27% of paid reviews had a 5 star rating


In [94]:
percent_unpaid = (u / unpaid_reviews_df) *100
print(f'{round(percent_unpaid, 2)}% of unpaid reviews had a 5 star rating')

61.62% of unpaid reviews had a 5 star rating
